In [1]:
from scripts.tools.issuetracker import *

In [7]:
def iter2(task, seen, level):
    for i in task.subtasks or ():
        seen.add(i)
        yield level+1, i
        yield from iter2(i, seen, level+1)

def iter_subtasks2(issues, seen=None, level=0):
    if seen is None:
        seen = set()
    for i in issues:
        if i.parent is not None or i in seen:
            continue
        seen.add(i)
        yield level, i
        yield from iter2(i, seen, level+1)
    if level == 0:
        assert all(i in seen for i in issues)

In [24]:
from officelib import wordlib
from officelib.wordlib import inches_to_points, c as wdconst

def format_paragraph(p, add_tabstops=True):
    p.Format.SpaceBefore = 0
    p.Format.SpaceBeforeAuto = False
    p.Format.SpaceAfter = 10
    p.Format.SpaceAfterAuto = False
    p.Format.LineUnitBefore = 0
    p.Format.LineUnitAfter = 0
    p.Format.LineSpacingRule = wdconst.wdLineSpaceSingle
    if add_tabstops:
        p.TabStops.Add(inches_to_points(6), wdconst.wdAlignTabLeft, wdconst.wdTabLeaderDots)

def _insert_lines(d, lines):
    pgs = d.Paragraphs
    p = pgs(pgs.Count)
    d.Style.NoSpaceBetweenParagraphsOfSameStyle = True
    r = p.Range
    for lvl, txt, href in lines:
        p=r.Paragraphs(1)
        format_paragraph(p)
        r.Text = txt + "\t"
        r.MoveStart(wdconst.wdCharacter, len(txt)+2)  # Past "\t"
        d.Hyperlinks.Add(r, href, "", href, "Link")
        r.InsertAfter("\r")
        r.MoveStart(wdconst.wdCharacter, len(href) + 10)  # Past "Link\r"

def insert_lines(d, lines):
    d.ScreenUpdating = False
    try:
        return _insert_lines(d, lines)
    finally:
        d.ScreenUpdating = True

In [ ]:
api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
api.login()

issues= api.download_issues('pbssoftware', modified_on=">=2016-6-01")
issues = list(issues.values())

def filter_cb(iss):
    return iss.sprint_milestone.name == "3.0" and iss.tracker.name == "Specification"

path = "pdfs3"
download_issues_pdf_multithread(api, issues, path, filter_cb)
gantt_issues = api.download_gantt('pbssoftware')

ss = []
lines = []
issues2 = {i for i in issues if i.sprint_milestone.name == "3.0" and i.tracker.name == 'Specification'}
for lvl, i in iter_subtasks2(gantt_issues):
    if i not in issues2:
        continue
    link = api._base_url + "/" + "issues/" + str(i.id)
    s="%d,Issue #%d: %s,%s\n" % (lvl, i.id, i.subject, link)
    lines.append((lvl, "Issue #%d: %s" % (i.id, i.subject), link))
    ss.append(s)

with open("pdfs3/issue_outline.txt", 'wb') as f:
    f.write(''.join(ss).encode('utf-8'))

In [26]:
word = wordlib.Word()
d = word.Documents.Add()
d.Paragraphs(1).Style.NoSpaceBetweenParagraphsOfSameStyle = True
insert_lines(d, lines)

In [25]:
for p in d.Paragraphs:
    format_paragraph(p, False)